In [7]:
import pandas as pd

In [8]:
sp = pd.read_csv('sp.csv')
madrid = pd.read_csv('madrid.csv')
br = pd.read_csv('brasil.csv', index_col=[0])

In [11]:
madrid

,Address,Price,Rooms,Squared_meters,Floor,Link,Summary,Distric,Type,Pool,Furniture,Exterior,Elevator
0,"calle Aguarón, Valdemarín, Madrid",900.0,NaN,55,2,www.idealista.com/inmueble/98588378/,\nChrystie & Didie ofrece en Exclusiva. Loft m...,Chamartin,Studio,False,False,True,True
1,"calle de Luisa Fernanda, Argüelles, Madrid",700.0,NaN,50,4,www.idealista.com/inmueble/91350410/,\nDisponible a principios de septiembre. Alqui...,Chamartin,Studio,False,False,False,True
2,"calle de Irún, Argüelles, Madrid",900.0,NaN,73,2,www.idealista.com/inmueble/98145156/,"\nExcelente estudio, junto a la estación de Pr...",Chamartin,Studio,False,False,True,True
3,"avenida del Doctor Federico Rubio y Galí, 88, ...",850.0,NaN,41,0,www.idealista.com/inmueble/98614584/,\nSe alquila estudio amueblado de 40 m2 recién...,Chamartin,Studio,False,True,True,True
4,"Valdemarín, Madrid",795.0,NaN,42,1,www.idealista.com/inmueble/97899849/,\nARAVACA-VALDEMARIN.\nExclusivo Loft de 40 M2...,Chamartin,Studio,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,"calle de San Restituto, 53, Valdezarza, Madrid",900.0,1.0,50,3,www.idealista.com/inmueble/92066241/,"\nEstupendo piso amueblado, con cocina totalme...",Moncloa,Flat,False,True,True,True
513,"calle Arriaza, 12, Argüelles, Madrid",900.0,1.0,50,3,www.idealista.com/inmueble/29286956/,"\nPrecioso apartamento de un dormitorio, salón...",Moncloa,Flat,False,False,False,True
514,"Aravaca, Madrid",990.0,2.0,80,0,www.idealista.com/inmueble/96987552/,"\nAravaca, estupendo amplio apartamento de 2 d...",Moncloa,Flat,False,False,True,True
515,"Aravaca, Madrid",995.0,2.0,75,0,www.idealista.com/inmueble/98349774/,"\nOCASION, se alquila apartamento en Aravaca, ...",Moncloa,Flat,False,False,True,True
